<a href="https://colab.research.google.com/github/magdalenacc/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [17]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [2]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss = 'categorical_crossentropy', 
              optimizer = 'Adam',
              metrics = ['accuracy'])
  
  logdir = os.path.join('logs', datetime.datetime.now()
  .strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(X_train,
            y_train,
            batch_size = params_fit.get('batch_size', 128),
            epochs = params_fit.get('epochs',5),
            verbose = params_fit.get('verbose', 1),
            validation_data=params_fit.get('validation_data', 
                                           (X_train, y_train)),
            callbacks = [tensorboard_callback]

            )
  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring = accuracy_score):
  y_test_norm = np.argmax(y_test, axis =1 )
  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis = 1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes ):
  return Sequential([
  Conv2D(filters = 32, kernel_size = (3,3), activation='relu',
         input_shape = input_shape),
  Conv2D(filters = 32, kernel_size = (3,3), activation='relu',
         padding='same'),        
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters = 64, kernel_size = (3,3), activation='relu', padding='same'),
  Conv2D(filters = 64, kernel_size = (3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters = 64, kernel_size = (3,3), activation='relu', padding='same'),
  Conv2D(filters = 64, kernel_size = (3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Flatten(),   
  Dense(1024, activation='relu'),
  Dropout(0.3),
  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(num_classes, activation='softmax'),                
])


In [9]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 5s 17ms/step - loss: 2.7039 - accuracy: 0.2776 - val_loss: 0.8306 - val_accuracy: 0.7364
Epoch 2/5
272/272 [==============================] - 4s 15ms/step - loss: 0.7386 - accuracy: 0.7694 - val_loss: 0.2021 - val_accuracy: 0.9403
Epoch 3/5
272/272 [==============================] - 4s 15ms/step - loss: 0.3694 - accuracy: 0.8847 - val_loss: 0.0853 - val_accuracy: 0.9744
Epoch 4/5
272/272 [==============================] - 4s 15ms/step - loss: 0.2540 - accuracy: 0.9234 - val_loss: 0.0687 - val_accuracy: 0.9805
Epoch 5/5
272/272 [==============================] - 4s 15ms/step - loss: 0.1768 - accuracy: 0.9474 - val_loss: 0.0387 - val_accuracy: 0.9879


0.9680272108843537

In [10]:
model_trained.evaluate(X_test, y_test)

138/138 [==============================] - 0s 4ms/step - loss: 0.1313 - accuracy: 0.9680


[0.1313130408525467, 0.9680272340774536]

In [0]:
def get_model(params ):
  return Sequential([
  Conv2D(filters = 32, kernel_size = (3,3), activation='relu',
         input_shape = input_shape),
  Conv2D(filters = 32, kernel_size = (3,3), activation='relu',
         padding='same'),        
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_one']),

  Conv2D(filters = 64, kernel_size = (3,3), activation='relu', padding='same'),
  Conv2D(filters = 64, kernel_size = (3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_two']),

  Conv2D(filters = 128, kernel_size = (3,3), activation='relu', padding='same'),
  Conv2D(filters = 128, kernel_size = (3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_three']),

  Flatten(),   
  Dense(1024, activation='relu'),
  Dropout(params['dropout_dense_block_one']),
  Dense(1024, activation='relu'),
  Dropout(params['dropout_dense_block_two']),

  Dense(num_classes, activation='softmax'),                
])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss = 'categorical_crossentropy', 
              optimizer = 'Adam',
              metrics = ['accuracy'])
  

  model.fit(X_train,
            y_train,
            batch_size = int(params.get('batch_size', 128)),
            epochs = 5,
            verbose = 0,
            )
  score = model.evaluate(X_test, y_test, verbose= 0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format( accuracy))
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [34]:
space={
    'batch_size': hp.quniform('batch_size', 100, 200,10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),


}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    25,
    Trials()
)

{'batch_size': 190.0, 'dropout_cnn_block_one': 0.381595309675514, 'dropout_cnn_block_three': 0.37555042053091636, 'dropout_cnn_block_two': 0.32727783930046406, 'dropout_dense_block_one': 0.34558624413662575, 'dropout_dense_block_two': 0.4081001687421748}
accuracy=0.9492063522338867
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.43831078928685563, 'dropout_cnn_block_three': 0.36445836028142203, 'dropout_cnn_block_two': 0.41695259188177336, 'dropout_dense_block_one': 0.616348543886317, 'dropout_dense_block_two': 0.4328658792859707}
accuracy=0.9369614720344543
{'batch_size': 170.0, 'dropout_cnn_block_one': 0.4106036897974073, 'dropout_cnn_block_three': 0.3277833832693585, 'dropout_cnn_block_two': 0.3585959374979007, 'dropout_dense_block_one': 0.4645441769523926, 'dropout_dense_block_two': 0.5453427981840591}
accuracy=0.9566893577575684
{'batch_size': 160.0, 'dropout_cnn_block_one': 0.3769209565713976, 'dropout_cnn_block_three': 0.39609512749478437, 'dropout_cnn_block_two': 0.33702691530